# 🧠 Text Generation using RNN on Shakespeare Dataset

**Goal:** Build a character-level RNN to generate text in the style of Shakespeare

### Load and Explore the Shakespeare Dataset

In [1]:
import tensorflow as tf

# Load the dataset from TensorFlow's repository
path_to_file = tf.keras.utils.get_file("shakespeare.txt", 
                                       "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

# Read the text
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Show the length of the text and a sample
print(f"Length of text: {len(text)} characters")
print(text[:500])  # Show the first 500 characters

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


### Preprocess the Data for RNN Training

In [2]:
import numpy as np
# Get unique characters in the dataset
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

# Creating a mapping from characters to numbers
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Convert all text to integers
text_as_int = np.array([char2idx[c] for c in text])

# Show the first 20 characters as integers
print(text[:20])
print(text_as_int[:20])

65 unique characters
First Citizen:
Befor
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


### Create Input Sequences and Targets

In [3]:
# Set the length of each sequence
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Convert stream of characters into sequences
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Function to split input and target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Preview the first batch
for input_example, target_example in dataset.take(1):
    print("Input:", ''.join(idx2char[input_example.numpy()]))
    print("Target:", ''.join(idx2char[target_example.numpy()]))

Input: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
Target: irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


### Create Training Batches

In [4]:
# Batch size: number of sequences processed together
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

# Final training dataset
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

### Build the RNN Model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Embedding, GRU, Dense

# Get unique characters and vocabulary size
vocab = sorted(set(text))
vocab_size = len(vocab)

# Hyperparameters
embedding_dim = 256
rnn_units = 1024

# Build model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        InputLayer(batch_input_shape=(batch_size, None), dtype='int32'),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, None, 256)             │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (64, None, 1024)            │       3,938,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, None, 65)              │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,021,569 (15.34 MB)

 Trainable params: 4,021,569 (15.34 MB)

 Non-trainable params: 0 (0.00 B)

### Compile the Model

In [6]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [9]:
# Enable Checkpointing for Resuming Training
# Directory for saving checkpoints
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

### Train the RNN Model

In [10]:
# Train for More Epochs with Callback
EPOCHS = 30  # or more if needed

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 406s 2s/step - loss: 3.0854
Epoch 2/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - loss: 1.9105
Epoch 3/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 401s 2s/step - loss: 1.6414
Epoch 4/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 444s 2s/step - loss: 1.5000
Epoch 5/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - loss: 1.4230
Epoch 6/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 408s 2s/step - loss: 1.3663
Epoch 7/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 436s 2s/step - loss: 1.3260
Epoch 8/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 421s 2s/step - loss: 1.2866
Epoch 9/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 403s 2s/step - loss: 1.2525
Epoch 10/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 410s 2s/step - loss: 1.2215
Epoch 11/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 403s 2s/step - loss: 1.1906
Epoch 12/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - loss: 1.1619
Epoch 13/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 406s 2s/step - loss: 1.1304
Epoch 14/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 402s 2s/step - loss: 1.0992
Epoch 15/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 40

### Save the Trained Model

In [11]:
model.save('shakespeare_rnn_model.keras')

### Load the Model

In [12]:
from tensorflow.keras.models import load_model

model = load_model('shakespeare_rnn_model.keras')

### ----------------------------------------------- **Model Completed** ----------------------------------------------------